conda env: TabPFN

In [ ]:
!git clone https://github.com/automl/TabPFN

In [1]:
import sys
import numpy as np
from pathlib import Path
import pandas as pd
import torch
import openml
import os
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pathlib import Path

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tabpfn_path = 'TabPFN'
sys.path.insert(0, tabpfn_path)
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

train = pd.read_csv('../data/train.csv', sep=",")
test = pd.read_csv('../data/test.csv')

import sklearn
y = train['SalePrice']
X = train.drop('SalePrice', axis=1)
categorical = [var for var in X.columns if X[var].dtype=='O']
numerical = [var for var in X.columns if X[var].dtype!='O']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical)
        ])
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32, base_path=tabpfn_path)        
pipeline = Pipeline(steps=[('imputation', preprocessor), ('classification', classifier)])


X[categorical] = X[categorical].fillna('None')


y = np.log1p(y)
mean_value = np.mean(y)
binarized_y = np.where(y > mean_value, 1, 0)
to_convert = {i: int for i,j in list(zip(X.dtypes.index, X.dtypes.values)) if j == 'bool'}
X = X.astype(to_convert)
# clf = cross_val_score(
#         pipeline,
#         X, binarized_y, scoring = 'accuracy'
# )
X_train, X_test, y_train, y_test = \
    train_test_split(X, binarized_y, random_state=42, test_size=0.35)

In [4]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputation',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvGr',
                                                   'KitchenAbvGr',
                                                   'TotRmsAbvGrd', 'Fireplaces',
                                                   'GarageYrBlt', 'GarageCars',
                                                   'GarageArea', 'WoodDeckSF',
                                                   'OpenPorchSF', ...])])),
                ('classification',
                 TabPFNClassifier(N_ensemble_configurations=32,
                                  base_path='TabPFN'))])

In [5]:
predictions = pipeline.predict(X_test)

/home/martin/miniconda3/envs/TabPFN/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [6]:
accuracy_score(y_test, predictions)

0.9530332681017613

In [19]:
import lightgbm
from sklearn.preprocessing import OneHotEncoder
lgbm = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.01, 
                                n_estimators=600, subsample_for_bin=20000, objective='binary', 
                                subsample=1.0, subsample_freq=0, colsample_bytree=1.0, bagging_freq=5,
                                bagging_fraction= 0.8, feature_fraction=0.9, metric='binary_logloss',
                                n_jobs=- 1, silent=False, importance_type='split')
categorical = [var for var in X.columns if X[var].dtype=='O']
numerical = [var for var in X.columns if X[var].dtype!='O']
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
lgbm_pipeline = Pipeline(steps=[('one-hot-encoding', categorical_transformer), 
                                ('classification', lgbm)])

# lgbm.fit(X_train, y_train)
# predictions = lgbm.predict(X_test)

In [20]:
lgbm_pipeline.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Number of positive: 461, number of negative: 488
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000596 seconds.
You can set `force_row_wise=true` to 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Pipeline(steps=[('one-hot-encoding',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                ('classification',
                 LGBMClassifier(bagging_fraction=0.8, bagging_freq=5,
                                feature_fraction=0.9, learning_rate=0.01,
                                metric='binary_logloss', n_estimators=600,
                                n_jobs=-1, objective='binary', silent=False,
                                subsample_for_bin=20000))])

In [21]:
predictions = lgbm_pipeline.predict(X_test)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


In [22]:
accuracy_score(y_test, predictions)

0.9099804305283757